# Praktikum 3

In [1]:
!pip install hnswlib

In [2]:
# membandingkan exact NN dengan HNSW pada 1000 data 2D.
import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

# ===========================
# 1. Buat data 2D acak
# ===========================
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

# ===========================
# 2. Exact NN (Brute Force)
# ===========================
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "detik")

# ===========================
# 3. HNSW
# ===========================
# Inisialisasi index HNSW
p = hnswlib.Index(space='l2', dim=dim)

# Ukuran maksimum elemen yang bisa ditampung
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Tambahkan data
p.add_items(data)

# Set parameter pencarian
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "detik")


=== Exact NN ===
Indices: [[828 565 234 137 911]]
Distances: [[0.01335504 0.01783607 0.02758248 0.03231153 0.03936457]]
Waktu: 0.20361709594726562 detik

=== HNSW ===
Indices: [[828 565 234 137 911]]
Distances: [[0.00017836 0.00031813 0.00076079 0.00104403 0.00154957]]
Waktu: 0.0002923011779785156 detik


c:\Users\sfati\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\sfati\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
        "wmic CPU Get NumberOfCores /Format:csv".split(),
        capture_output=True,
        text=True,
    )
  File "c:\Users\sfati\anaconda3\Lib\subprocess.py", line 554, in run
    with Popen(*popenargs, **kwargs) as process:
         ~~~~~^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\sfati\anaconda3\Lib\subprocess.py", line 1039, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^

| Metric     | Dimensi | Jumlah Data |     Waktu Exact (s)     |     Waktu HNSW (s)     |  Rasio Kecepatan (Exact/HNSW) |
|-------------|----------|--------------|---------------------------|---------------------------|-------------------------------|
| euclidean  | 2D| 1000 |0002163 |  0.000257 |  8.42 |
| euclidean  | 2D | 1000000 |  0.382114  |0.016890  |   22.63  |
| euclidean  | 5D | 1000  |  0.002154 |  0.000244 | 8.83 |
| euclidean  | 5D | 1000000| 0.380114 |  0.017123  |22.19 |
| cosine     | 2D | 1000  | 0.002107 | 0.000255 |  8.26  |
| cosine     | 2D| 1000000 |  0.384912| 0.016754  |  22.96
| cosine     | 5D  | 1000 | 0.002104|   0.000238  | 8.84   |
| cosine     | 5D  | 1000000 |  0.382918  | 0.017004   |22.52  |